In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 🌸 **ScentMatch AI - Your Personal Perfume Recommender(Capstone Project)**

This notebook is my capstone project for the **5-Day AI Agents Intensive (Google × Kaggle)**.  
I built an ADK-powered agent that helps users pick the perfect perfume based on **vibe, season, gender, usage, and budget** - and even suggests **dupes for expensive fragrances**.



## 🎯 **Project Goal**

ScentMatch AI helps users:

* 👃 **Discover perfumes** based on their preferences (gender, season, occasion, vibe, budget)
* 💸 **Find cheaper alternatives** to expensive fragrances (example: “dupes” for Creed Aventus)
* 💧 **Get usage recommendations** such as sprays/day, longevity expectations, and bottle lifetime

The agent uses:

* **Google Gemini** (via `google-adk`) for reasoning + conversation  
* **Custom function tools** for structured fragrance recommendations  
* **In-memory sessions** so it remembers context through a conversation  



# 🚀 **Why This Project Matters**

Choosing a perfume is surprisingly complex:

* Different scents work in different **climates**
* Prices range from **budget to luxury niche**
* Marketing descriptions are often vague
* Many people look for **affordable alternatives (dupes)**
* Beginners don’t understand **projection, longevity, or strength**

ScentMatch AI acts like a friendly, knowledgeable assistant that:

* Understands what the user likes  
* Filters scents intelligently  
* Explains *why* it made each recommendation  
* Gives practical, everyday usage tips  

It’s like having a personal fragrance expert - powered by ADK.



# 🧠 **ADK Concepts Demonstrated**

## ✔️ 1. `LlmAgent`

Handles conversation and decides when to call the right tool.

## ✔️ 2. Function Tools (Custom Tools)

These tools power the actual “intelligence” of ScentMatch AI:

### 🔧 **Tools Used in This Agent**

| Tool                       | What it Does                                                                      |
| -------------------------- | --------------------------------------------------------------------------------- |
| **`recommend_perfumes()`** | Matches perfumes using multi-filter logic: gender, season, occasion, vibe, budget |
| **`find_alternative()`**   | Finds cheaper perfumes with similar vibes + calculates savings                    |
| **`get_usage_tips()`**     | Suggests sprays/day, climate adjustments, 100 ml bottle lifetime estimate         |

All tools follow ADK’s structured input/output patterns.

## ✔️ 3. InMemory Sessions

Allows multi-turn conversations with memory.  
Example: user first asks for *“a fresh summer fragrance”* → next turn they say *“give me a cheaper one”* → the agent understands the reference and keeps context.

## ✔️ 4. InMemoryRunner + Streaming

Handles agent execution and streams responses in real time using `run_async()`.



# 🧩 **Architecture Overview**

Here’s the full flow of how ScentMatch AI works:

```text
                        ┌───────────────────┐
                        │     User Query    │
                        └─────────┬─────────┘
                                  │
                                  ▼
                     ┌───────────────────────────┐
                     │         LlmAgent          │
                     │  (Gemini 2.5 Flash Lite)  │
                     └─────────┬─────────────────┘
                               │
           ┌───────────────────┼─────────────────────┐
           │                   │                     │
           ▼                   ▼                     ▼
 ┌──────────────────┐ ┌────────────────────┐ ┌──────────────────────┐
 │ recommend_       │ │ find_alternative() │ │   get_usage_tips()   │
 │ perfumes()       │ │   (cheaper dupes)  │ │ (sprays/day, bottle  │
 │  (main match)    │ │                    │ │  life, climate)      │
 └─────────┬────────┘ └──────────┬─────────┘ └──────────┬───────────┘
           │                     │                      │
           └───────────────┬─────┴──────────────┬───────┘
                           ▼                    ▼
                    ┌─────────────────────────────────┐
                    │ Structured Output (Python dict) │
                    │   → Runner → Notebook reply     │
                    └─────────────────────────────────┘


# 🛠️ Setup & Environment

Before building the agent, we install all required packages and load our `GOOGLE_API_KEY` securely using Kaggle Secrets.

Steps in this section:

1. Install `google-adk` (usually pre-installed in course environment)
2. Load API key from Kaggle Secrets
3. Import ADK components (agents, sessions, runners, tools)
4. Prepare data structures used by the tools

Let’s start! 👇


In [24]:
# 1. Install Google ADK (already available in course env, but safe to keep)
%pip install -q google-adk
print("✅ google-adk installed (or already available).")


Note: you may need to restart the kernel to use updated packages.
✅ google-adk installed (or already available).


In [25]:
# 2. Load GOOGLE_API_KEY from Kaggle Secrets and set env variable

import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(f"❌ Authentication Error: Please make sure you added 'GOOGLE_API_KEY' to Kaggle secrets. Details: {e}")


✅ Setup and authentication complete.


In [26]:
# 3. Import ADK components

from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import FunctionTool, ToolContext

print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


# 🌺 Perfume Catalog (Custom Dataset)

To enable meaningful recommendations, I built a **structured perfume catalog** that includes:

* Name  
* Brand  
* Gender  
* Season suitability  
* Occasion  
* Vibe keywords  
* Projection & longevity  
* Price level + estimated price  

This dataset powers all the custom tools below.


In [27]:
# 4. Simple perfume catalog used by our tools

from typing import List, Dict, Any, Optional

Perfume = Dict[str, Any]

PERFUMES: List[Perfume] = [
    {
        "name": "Creed Aventus",
        "brand": "Creed",
        "gender": "masculine",
        "season": ["spring", "summer", "autumn"],
        "occasion": ["date", "party", "special"],
        "vibe": ["fruity", "smoky", "boss", "attention-grabbing"],
        "strength": "eau de parfum",
        "longevity": "8-10h",
        "projection": "strong",
        "price_level": "luxury",
        "price_estimate_usd": 350,
        "link": "https://www.creedboutique.com/products/aventus",
    },
    {
        "name": "Armaf Club de Nuit Intense Man",
        "brand": "Armaf",
        "gender": "masculine",
        "season": ["spring", "autumn", "winter"],
        "occasion": ["party", "night out"],
        "vibe": ["fruity", "smoky", "beast-mode", "aventus-style"],
        "strength": "eau de toilette",
        "longevity": "7-9h",
        "projection": "strong",
        "price_level": "budget",
        "price_estimate_usd": 45,
        "link": "https://www.armafperfume.com/",
    },
    {
        "name": "Dior Sauvage EDT",
        "brand": "Dior",
        "gender": "masculine",
        "season": ["all"],
        "occasion": ["daily", "date", "office"],
        "vibe": ["fresh", "clean", "mass-appeal", "blue"],
        "strength": "eau de toilette",
        "longevity": "6-8h",
        "projection": "moderate-strong",
        "price_level": "designer",
        "price_estimate_usd": 120,
        "link": "https://www.dior.com/",
    },
    {
        "name": "Nautica Voyage",
        "brand": "Nautica",
        "gender": "masculine",
        "season": ["spring", "summer"],
        "occasion": ["daily", "casual"],
        "vibe": ["fresh", "aquatic", "soapy", "cheap-gem"],
        "strength": "eau de toilette",
        "longevity": "4-6h",
        "projection": "moderate",
        "price_level": "budget",
        "price_estimate_usd": 25,
        "link": "https://www.nautica.com/",
    },
    {
        "name": "Jean Paul Gaultier Le Male Le Parfum",
        "brand": "Jean Paul Gaultier",
        "gender": "masculine",
        "season": ["autumn", "winter"],
        "occasion": ["date", "party", "club"],
        "vibe": ["sweet", "vanilla", "sexy", "warm"],
        "strength": "eau de parfum",
        "longevity": "8-10h",
        "projection": "strong",
        "price_level": "designer",
        "price_estimate_usd": 120,
        "link": "https://www.jeanpaulgaultier.com/",
    },
        {
        "name": "Bleu de Chanel EDT",
        "brand": "Chanel",
        "gender": "masculine",
        "season": ["all"],
        "occasion": ["office", "daily", "date"],
        "vibe": ["fresh", "blue", "clean", "mass-appeal"],
        "strength": "eau de toilette",
        "longevity": "6-8h",
        "projection": "moderate",
        "price_level": "designer",
        "price_estimate_usd": 150,
        "link": "https://www.chanel.com/",
    },
    {
        "name": "Prada Luna Rossa Carbon",
        "brand": "Prada",
        "gender": "masculine",
        "season": ["all"],
        "occasion": ["daily", "office", "gym"],
        "vibe": ["fresh", "clean", "metallic", "modern"],
        "strength": "eau de toilette",
        "longevity": "6-7h",
        "projection": "moderate",
        "price_level": "designer",
        "price_estimate_usd": 110,
        "link": "https://www.prada.com/",
    },
    {
        "name": "Versace Dylan Blue",
        "brand": "Versace",
        "gender": "masculine",
        "season": ["all"],
        "occasion": ["daily", "party"],
        "vibe": ["fresh", "blue", "sexy", "clean"],
        "strength": "eau de toilette",
        "longevity": "6-8h",
        "projection": "moderate",
        "price_level": "designer",
        "price_estimate_usd": 95,
        "link": "https://www.versace.com/",
    },
    {
        "name": "Burberry Hero EDT",
        "brand": "Burberry",
        "gender": "masculine",
        "season": ["spring", "summer"],
        "occasion": ["daily", "casual"],
        "vibe": ["fresh", "woody", "light", "clean"],
        "strength": "eau de toilette",
        "longevity": "4-6h",
        "projection": "moderate",
        "price_level": "designer",
        "price_estimate_usd": 90,
        "link": "https://us.burberry.com/",
    },
    {
        "name": "Lattafa Asad",
        "brand": "Lattafa",
        "gender": "masculine",
        "season": ["autumn", "winter"],
        "occasion": ["date", "party"],
        "vibe": ["vanilla", "spicy", "warm", "sexy"],
        "strength": "eau de parfum",
        "longevity": "8-12h",
        "projection": "strong",
        "price_level": "budget",
        "price_estimate_usd": 25,
        "link": "https://www.lattafa.com/",
    },
    {
        "name": "Lattafa Yara",
        "brand": "Lattafa",
        "gender": "feminine",
        "season": ["all"],
        "occasion": ["daily", "date"],
        "vibe": ["sweet", "vanilla", "fruity", "creamy"],
        "strength": "eau de parfum",
        "longevity": "6-10h",
        "projection": "moderate-strong",
        "price_level": "budget",
        "price_estimate_usd": 30,
        "link": "https://www.lattafa.com/",
    },
    {
        "name": "Carolina Herrera Bad Boy",
        "brand": "Carolina Herrera",
        "gender": "masculine",
        "season": ["autumn", "winter"],
        "occasion": ["party", "date"],
        "vibe": ["sweet", "sexy", "warm", "amber"],
        "strength": "eau de toilette",
        "longevity": "7-9h",
        "projection": "moderate-strong",
        "price_level": "designer",
        "price_estimate_usd": 110,
        "link": "https://www.carolinaherrera.com/",
    },
    {
        "name": "Mont Blanc Explorer",
        "brand": "Mont Blanc",
        "gender": "masculine",
        "season": ["all"],
        "occasion": ["daily", "office", "casual"],
        "vibe": ["fresh", "fruity", "aventus-style"],
        "strength": "eau de parfum",
        "longevity": "6-8h",
        "projection": "moderate",
        "price_level": "designer",
        "price_estimate_usd": 85,
        "link": "https://www.montblanc.com/",
    },
        {
        "name": "Jean Paul Gaultier Le Male Elixir Absolu",
        "brand": "Jean Paul Gaultier",
        "gender": "masculine",
        "season": ["autumn", "winter", "night"],
        "occasion": ["date", "night-out", "party", "club"],
        "vibe": ["sweet", "spicy", "vanilla", "warm", "sexy"],
        "strength": "eau de parfum",
        "longevity": "9-11h",
        "projection": "strong-beast",
        "price_level": "designer",
        "price_estimate_usd": 130,
        "link": "https://www.jeanpaulgaultier.com",
    },
    {
        "name": "Mancera Cedrat Boise",
        "brand": "Mancera",
        "gender": "masculine",
        "season": ["all"],
        "occasion": ["daily", "office", "casual", "date"],
        "vibe": ["fresh", "citrus", "woody", "versatile"],
        "strength": "eau de parfum",
        "longevity": "8-9h",
        "projection": "moderate-strong",
        "price_level": "high",
        "price_estimate_usd": 195,
        "link": "https://www.mancera-paris.com",
    },
    
]
print(f"✅ Loaded perfume catalog with {len(PERFUMES)} perfumes.")


✅ Loaded perfume catalog with 15 perfumes.


# 🔧 Custom Tools Implementation

Below are the core function tools powering ScentMatch AI:

* `recommend_perfumes()` → Matches scents using multi-filter ranking logic  
* `find_alternative()` → Finds cheaper dupes with overlapping vibe tokens  
* `get_usage_tips()` → Calculates sprays/day + bottle lifetime  

Each tool follows ADK’s structured input/output pattern to ensure the agent can call them reliably.


In [28]:
# 5. Define ScentMatch AI tools

def _normalize(s: str | None) -> str:
    return s.strip().lower() if isinstance(s, str) else ""


def _matches_pref(value: str | None, options: list[str] | None) -> bool:
    """
    Check if a preference fits a list.

    - Accepts comma-, slash-, or space-separated input ("fresh, clean long lasting")
    - Case-insensitive and partial matching.
    """
    if not value:
        return True
    if not options:
        return True

    value = _normalize(value)

    # Split on commas, slashes, and spaces
    raw = value.replace("/", " ").replace(",", " ")
    tokens = [t.strip() for t in raw.split() if t.strip()]

    options_norm = [_normalize(o) for o in options]

    # if ANY token appears in ANY option, we treat it as a match
    return any(tok in opt for tok in tokens for opt in options_norm)


def _price_level_key(level: str) -> int:
    """
    Simple ranking for price_level.
    Lower = cheaper.
    """
    order = {
        "budget": 0,
        "designer": 1,
        "high": 2,
        "niche": 3,
        "luxury": 4,
    }
    return order.get(_normalize(level), 2)


def recommend_perfumes(
    gender: Optional[str] = None,
    occasion: Optional[str] = None,
    season: Optional[str] = None,
    vibe: Optional[str] = None,
    budget_level: Optional[str] = None,
) -> dict:
    """
    Recommend perfumes based on user preferences.
    """
    try:
        g = _normalize(gender)
        b = _normalize(budget_level)

        matches: list[Perfume] = []
        for p in PERFUMES:
            # gender check (allow "all" or "unisex")
            if g:
                pg = _normalize(p.get("gender", ""))
                if pg not in ("all", "unisex"):
                    if g not in pg:
                        continue

            # season / occasion / vibe checks
            if season and not _matches_pref(season, p.get("season")):
                continue
            if occasion and not _matches_pref(occasion, p.get("occasion")):
                continue
            if vibe and not _matches_pref(vibe, p.get("vibe")):
                continue

            # budget check (soft): only keep perfumes <= requested level
            if b:
                if _price_level_key(p.get("price_level", "")) > _price_level_key(b):
                    continue

            matches.append(p)

        # If nothing strict matches, fallback to simple "good picks"
        if not matches:
            fallback = sorted(
                PERFUMES,
                key=lambda x: x.get("price_estimate_usd", 9999),
            )[:5]

            return {
                "status": "fallback",
                "note": (
                    "I couldn't find a perfect match, so here are some popular "
                    "everyday options instead."
                ),
                "data": fallback,
            }

        # sort: cheaper first, then stronger projection
        projection_rank = {
            "soft": 0,
            "moderate": 1,
            "moderate-strong": 2,
            "strong": 3,
            "strong-beast": 4,
        }

        matches_sorted = sorted(
            matches,
            key=lambda x: (
                x.get("price_estimate_usd", 9999),
                -projection_rank.get(_normalize(x.get("projection", "")), 1),
            ),
        )

        return {
            "status": "success",
            "data": matches_sorted[:5],  # top 5
        }

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


def find_alternative(
    reference_name: str,
    max_price_level: Optional[str] = None,
) -> dict:
    """
    Find cheaper / similar alternatives for a reference perfume.
    """
    try:
        ref_name_norm = _normalize(reference_name)
        max_level_norm = _normalize(max_price_level)

        # 1) Try to find the reference perfume in our catalog
        ref = None
        for p in PERFUMES:
            if ref_name_norm in _normalize(p["name"]):
                ref = p
                break

        if not ref:
            # Fallback: reference not in catalog → just give good budget picks
            fallback = sorted(
                PERFUMES,
                key=lambda x: x.get("price_estimate_usd", 9999),
            )[:3]

            return {
                "status": "fallback",
                "error_message": (
                    "That exact perfume is not in my small catalog. "
                    "Here are some popular, affordable options instead."
                ),
                "reference_name": reference_name,
                "data": fallback,
            }

        ref_vibes = set(_normalize(v) for v in ref.get("vibe", []))

        candidates = []
        for p in PERFUMES:
            if p["name"] == ref["name"]:
                continue

            # similar vibe overlap
            overlap = ref_vibes.intersection(_normalize(v) for v in p.get("vibe", []))
            if not overlap:
                continue

            # cheaper?
            if p.get("price_estimate_usd", 9999) >= ref.get("price_estimate_usd", 9999):
                continue

            # respect user price cap if given
            if max_level_norm:
                if _price_level_key(p.get("price_level", "")) > _price_level_key(max_level_norm):
                    continue

            saving = ref.get("price_estimate_usd", 0) - p.get("price_estimate_usd", 0)
            candidates.append((len(overlap), saving, p))

        if not candidates:
            return {
                "status": "no_results",
                "reference": ref,
                "error_message": "No cheaper alternatives found in this tiny demo list.",
            }

        # sort: more vibe overlap first, then more saving
        candidates_sorted = [
            c[-1] for c in sorted(candidates, key=lambda x: (-x[0], -x[1]))
        ]

        return {
            "status": "success",
            "reference": ref,
            "data": candidates_sorted[:3],
        }

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


def get_usage_tips(
    perfume_name: str,
    sprays_per_day: Optional[int] = None,
    climate: Optional[str] = None,
) -> dict:
    """
    Give practical usage tips for a perfume.
    """
    try:
        search_name = _normalize(perfume_name)

        p = None
        for item in PERFUMES:
            if search_name in _normalize(item["name"]):
                p = item
                break

        if not p:
            return {
                "status": "no_results",
                "error_message": (
                    "That perfume is not in my small catalog yet. "
                    "Try another one or ask for a recommendation first."
                ),
            }

        projection = _normalize(p.get("projection", "moderate"))
        # rough rule of thumb
        if projection in ["strong", "strong-beast", "moderate-strong"]:
            default_sprays = 4
        elif projection == "soft":
            default_sprays = 6
        else:
            default_sprays = 5

        climate_norm = _normalize(climate)

        if "hot" in climate_norm:
            recommended_sprays = max(2, default_sprays - 2)
        elif "cold" in climate_norm:
            recommended_sprays = default_sprays + 1
        else:
            recommended_sprays = default_sprays

        if sprays_per_day is None:
            sprays_per_day = recommended_sprays

        # estimate bottle life for 100ml, 1 spray ≈ 0.1ml → 1000 sprays
        estimated_days = round(1000 / max(1, sprays_per_day))

        tips = {
            "perfume": p["name"],
            "brand": p["brand"],
            "recommended_sprays": recommended_sprays,
            "user_sprays": sprays_per_day,
            "projection": p["projection"],
            "longevity": p["longevity"],
            "estimated_days_for_100ml": estimated_days,
            "note": (
                "These are rough community-style estimates based on projection and climate, "
                "not exact lab measurements."
            ),
        }

        return {"status": "success", "data": tips}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}


# 🤖 Building the LlmAgent

Here, I configure the **LlmAgent** using:

* Gemini 2.5 Flash Lite  
* Three registered tools  
* Short system prompt explaining its role  
* In-memory session mode  

This agent is the “brain” of ScentMatch AI.



# 🧩 Registering Tools with the Agent

Each custom tool is registered here with:

* Name  
* Function  
* Input/output JSON schema  
* Description  

This allows the LlmAgent to decide which tool to call based on user input.


# 🚀 Running the Agent (InMemoryRunner)

I use `InMemoryRunner` to:

* Maintain conversation context  
* Execute tool calls  
* Stream responses in real-time  
* Return structured outputs  

This gives ScentMatch AI its interactive, chat-like behaviour.


In [29]:
# 6. Register tools and create the ScentMatch AI agent

perfume_tools = [
    FunctionTool(func=recommend_perfumes),
    FunctionTool(func=find_alternative),
    FunctionTool(func=get_usage_tips),
]

scentmatch_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite"),
    name="scentmatch_ai",
    description=(
        "A perfume expert agent that helps users pick fragrances, find cheaper alternatives, "
        "and plan daily usage based on their style, climate, and budget."
    ),
    tools=perfume_tools,
)

APP_NAME = "scentmatch_app"
USER_ID = "demo-user"

# ✅ InMemoryRunner now only needs app_name + agent
runner = InMemoryRunner(
    app_name=APP_NAME,
    agent=scentmatch_agent,
)

# (optional) if you still want a separate variable:
session_service = runner.session_service

print("✅ ScentMatch AI agent & runner initialized.")


✅ ScentMatch AI agent & runner initialized.


In [30]:
# 7. Helper function to talk to the agent (clean version)

import asyncio

async def run_session(
    runner_instance: InMemoryRunner,
    user_queries: list[str] | str,
    session_name: str = "default-session",
):
    print(f"\n### Session: {session_name}")

    app_name = runner_instance.app_name
    session_service_local = runner_instance.session_service

    # create or get session
    try:
        session = await session_service_local.create_session(
            app_name=app_name,
            user_id=USER_ID,
            session_id=session_name,
        )
    except Exception:
        session = await session_service_local.get_session(
            app_name=app_name,
            user_id=USER_ID,
            session_id=session_name,
        )

    if isinstance(user_queries, str):
        user_queries = [user_queries]

    for query in user_queries:
        print(f"\n👤 User > {query}")
        content = types.Content(role="user", parts=[types.Part(text=query)])

        async for event in runner_instance.run_async(
            user_id=USER_ID,
            session_id=session.id,
            new_message=content,
        ):
            # skip empty events
            if not getattr(event, "content", None):
                continue

            parts = getattr(event.content, "parts", None)
            if not parts:
                continue

            for part in parts:
                txt = getattr(part, "text", None)
                if txt and txt != "None":
                    print(f"🤖 ScentMatch AI > {txt}")

print("✅ Clean helper function loaded.")


✅ Clean helper function loaded.


# 💬 Demo: Chatting With ScentMatch AI

Below are three example conversations that demonstrate how ScentMatch AI:

✓ Understands user preferences  
✓ Selects and calls the correct tools  
✓ Returns structured, helpful recommendations  
✓ Maintains context across the conversation  

### **Example Sessions**

1. **Daily fresh fragrance** for a student in a hot climate (budget / cheap designer)  
2. **Cheaper alternatives (“dupes”)** for Creed Aventus  
3. **Sweet, sexy winter fragrances** for dates and parties (masculine, budget-friendly)


In [31]:
await run_session(
    runner,
    "I’m a guy in a hot climate, want a daily perfume for university and part-time job. "
    "Budget level is budget or cheap designer. I like fresh, clean, long lasting scents.",
    session_name="demo-session-01-clean",
)

await run_session(
    runner,
    "Creed Aventus is too expensive for me. Suggest cheaper alternatives with a similar vibe. "
    "Explain why they are similar and how much cheaper they are approximately.",
    session_name="demo-session-02-clean",
)

await run_session(
    runner,
    "I want a sweet, sexy winter perfume for parties and dates. Masculine. Budget or cheap designer. Give me 3 options.",
    session_name="final-test-session-clean",
)



### Session: demo-session-01-clean

👤 User > I’m a guy in a hot climate, want a daily perfume for university and part-time job. Budget level is budget or cheap designer. I like fresh, clean, long lasting scents.


🤖 ScentMatch AI > The user is a guy in a hot climate looking for a daily perfume for university and a part-time job with a budget of "budget or cheap designer". He likes fresh, clean, and long-lasting scents.

Here are a few popular everyday options that fit your criteria:

*   **Nautica Voyage**: This is a budget-friendly aquatic and fresh scent that's perfect for hot climates and daily wear. It has a moderate longevity of 4-6 hours and a moderate projection.

*   **Mont Blanc Explorer**: While slightly higher in price (designer level), this is a great option if you're willing to stretch your budget a bit. It has a fresh, fruity vibe, is long-lasting (6-8 hours), and suitable for all seasons, including hot climates. It's also a good choice for daily wear, office, and casual occasions.

Since you mentioned "budget or cheap designer," I've included both a budget-friendly option and a slightly more premium one. Nautica Voyage is a classic budget choice for fresh scents, and Mont Blanc Ex

🤖 ScentMatch AI > The most recommended alternative for Creed Aventus is Armaf Club de Nuit Intense Man. It is significantly cheaper, with an estimated price of around $45, while Creed Aventus costs about $350. Both fragrances share a similar "aventus-style" vibe, characterized by fruity and smoky notes, and are suitable for spring, autumn, and winter. Armaf Club de Nuit Intense Man is an Eau de Toilette with strong projection and longevity of 7-9 hours.

Another option is Mont Blanc Explorer, priced at approximately $85. It's also an Eau de Parfum with moderate projection and longevity of 6-8 hours. It falls into the designer price range and is suitable for all seasons, making it a versatile choice for daily wear, office, or casual occasions. It shares the fruity and fresh notes with Creed Aventus, offering a similar vibe.

Lattafa Yara is also listed as a budget-friendly option but is geared towards a feminine audience with a sweet, vanilla, and fruity vibe, so it might not be what yo

🤖 ScentMatch AI > I couldn't find a perfect match with those exact criteria. However, here are some popular options that are a great fit for a variety of occasions and might be what you're looking for:

1.  **Lattafa Asad (Budget):** This fragrance is an excellent choice for winter, with a warm, spicy, and vanilla-heavy profile that's perfect for dates and parties. It has strong projection and longevity.

2.  **Armaf Club de Nuit Intense Man (Budget):** While often compared to a more expensive niche fragrance, this is a versatile option that works well for parties and nights out. It has a fruity and smoky scent profile with strong performance, making it suitable for cooler seasons.

3.  **Mont Blanc Explorer (Designer):** This is a great all-around fragrance that leans towards fresh and fruity, but has enough depth for winter. It's suitable for daily wear, office, and casual occasions, with moderate projection and good longevity.


In [33]:
# 🔥 LIVE CHAT with ScentMatch AI (improved)

print("🌸 Chat with ScentMatch AI is ready!")
print("Type your perfume questions below.")
print("You can end the chat any time with:  exit / quit / end chat\n")

while True:
    query = input("You: ")

    normalized = query.strip().lower()

    # --- manual end commands BEFORE sending to agent ---
    if normalized in ["exit", "quit", "end chat"]:
        print("👋 ScentMatch AI: Thanks for chatting! Hope your next scent is a banger. Bye! ✨")
        break

    # send message to the agent in the same live session
    await run_session(
        runner,
        query,
        session_name="live-chat",   # keeps conversation context
    )

    # --- auto-end on natural goodbyes / thanks AFTER answering ---
    goodbye_triggers = [
        "bye",
        "goodbye",
        "see you",
        "see ya",
        "thank you",
        "thanks",
        "thx",
        "good night",
        "goodnite",
    ]

    if any(trigger in normalized for trigger in goodbye_triggers):
        print("\n👋 ScentMatch AI: Glad I could help! Ending this chat here. Smell amazing, see you next time 💫")
        break


🌸 Chat with ScentMatch AI is ready!
Type your perfume questions below.
You can end the chat any time with:  exit / quit / end chat



You:  i want a perfume good for winter



### Session: live-chat

👤 User > i want a perfume good for winter


🤖 ScentMatch AI > I couldn't find a perfect match, but here are some popular everyday options that might work for you. Lattafa Asad is a great option for winter with a long-lasting scent. Armaf Club de Nuit Intense Man is also a good choice, suitable for all seasons and has a strong projection. Mont Blanc Explorer is another good option, especially for casual wear.


You:  anything under 100$ 


...


### Session: live-chat

👤 User > which is the long lasting one
🤖 ScentMatch AI > Lattafa Asad has a longevity of 8-12 hours, Armaf Club de Nuit Intense Man has a longevity of 7-9 hours, and Mont Blanc Explorer has a longevity of 6-8 hours. Lattafa Asad is the longest lasting among the three.


You:  nice



### Session: live-chat

👤 User > nice
🤖 ScentMatch AI > Great! If you have any more questions or need further recommendations, feel free to ask.


You:  no thanks



### Session: live-chat

👤 User > no thanks
🤖 ScentMatch AI > Okay, have a great day!

👋 ScentMatch AI: Glad I could help! Ending this chat here. Smell amazing, see you next time 💫


# 🌟 Final Summary

ScentMatch AI is a complete ADK-powered agent that shows how LLMs + tools can solve a real consumer problem.  
It combines:

✔ A curated fragrance dataset  
✔ Intelligent tool functions (recommendation ◦ dupes ◦ usage tips)  
✔ Reasoning via Gemini 2.5 Flash Lite  
✔ Multi-turn conversations with InMemoryRunner  
✔ Clean, structured responses ideal for downstream apps  

### 🧠 Key Learnings
This capstone helped me understand:

* How to design meaningful, domain-focused tools  
* How an LlmAgent selects tools based on user intent  
* How to stream responses + manage sessions using ADK  
* How to structure agent workflows cleanly inside a notebook  
* How to build a real-world mini product with ADK

### 🚀 What’s Next (If I Continue This Project)

* Expanding the catalog (niche, designer, clones, Arabic perfumes)  
* Adding a lightweight vector search for better “vibe similarity”  
* Building a UI with Streamlit/Gradio  
* Turning this into a Telegram/WhatsApp mini-bot  
* Personalizing recommendations using user fragrance history  
* Adding a “compare perfumes” tool  

Thanks for checking out my project! 🌸✨
